In [27]:
import subprocess

In [44]:
"""
f = open(filename,"r");
lines = f.readlines();
for i in lines:
thisline = i.split(" ");
"""
a=subprocess.run(["build/oi"],input = b"10",capture_output=True)

In [52]:
print((a.stdout).decode("utf-8"))


	width : 
	size : 
	mass : 
	speedx : 
	speedy : 
	posx : 
	posy : c_time: 0
pos(x,y)10 5
speed(x,y)4.02 0
c_time: 1
pos(x,y)14.02 5
speed(x,y)3.04 0
c_time: 2
pos(x,y)17.06 5
speed(x,y)2.06 0
c_time: 3
pos(x,y)19.12 5
speed(x,y)1.08 0
c_time: 4
pos(x,y)20.2 5
speed(x,y)0.1 0
c_time: 5
All squares have stoped

